## Resource

- https://towardsdatascience.com/synthetic-data-generation-using-conditional-gan-45f91542ec6b

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Activation, BatchNormalization, Concatenate, Dense, Dropout, Multiply, Embedding, Flatten, Input, Reshape, LeakyReLU, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

In [9]:
# Define important parameters
img_shape = (700, 460, 1)
z_dim = 100
n_class = 2

In [10]:
# Generator CNN model
def generator_model(z_dim):

    model = Sequential()

    model.add(Dense(256 * 7 * 7, input_dim=z_dim,))
    model.add(Reshape((7, 7, 256)))

    model.add(Conv2DTranspose(128, 3, 2, padding='same',))
    model.add(LeakyReLU(alpha = 0.01))

    model.add(Conv2DTranspose(64, 3, 1, padding='same',))
    model.add(LeakyReLU(alpha = 0.01))

    model.add(Conv2DTranspose(1, 3, 2, padding='same',))
    model.add(LeakyReLU(alpha = 0.01))

    return model

# generator input 
def generator(z_dim):
    # latent input
    z = Input(shape=(z_dim, ))
    # label input
    label = Input(shape=(1, ), dtype='int32')
    # convert label to embedding
    label_embedding = Embedding(n_class, z_dim)(label)

    label_embedding = Flatten()(label_embedding)
    # dot product two inputs
    joined_representation = Multiply()([z, label_embedding])

    generator = generator_model(z_dim)

    conditioned_img = generator(joined_representation)

    model =  Model([z, label], conditioned_img)
    
    # save model blueprint to image
    # plot_model(model,'generator.jpg',show_shapes=True,show_dtype=True)

    return model

In [11]:
# discriminator CNN model
def discriminator_model(img_shape):

    model = Sequential()

    model.add(Conv2D(64,3,2,input_shape=(img_shape[0], img_shape[1], img_shape[2] + 1),))
    model.add(LeakyReLU(alpha = 0.01))

    model.add(Conv2D(64,3,2,input_shape=img_shape,padding='same',))
    model.add(LeakyReLU(alpha = 0.001))

    model.add(Conv2D(128,3,2,input_shape=img_shape,padding='same',))
    model.add(LeakyReLU(alpha = 0.001))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model


def discriminator(img_shape):
    # image input
    img = Input(shape=img_shape)
    # label input
    label = Input(shape=(1, ), dtype='int32')

    label_embedding = Embedding(n_class, np.prod(img_shape),input_length=1)(label)

    label_embedding = Flatten()(label_embedding)

    label_embedding = Reshape(img_shape)(label_embedding)
    # concatenate the image and label
    concatenated = Concatenate(axis=-1)([img, label_embedding])

    discriminator = discriminator_model(img_shape)

    classification = discriminator(concatenated)

    model = Model([img, label], classification)

    # plot_model(model,'discriminator.jpg',show_shapes=True,show_dtype=True)

    return model

In [12]:
# define a complete GAN architecture
def cgan(generator, discriminator):

    z = Input(shape=(z_dim, ))

    label = Input(shape=(1, ))

    img = generator([z, label])

    classification = discriminator([img, label])

    model = Model([z, label], classification)
    
    return model

In [13]:
discriminator = discriminator(img_shape)
# compile the discriminator architecture 
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

generator = generator(z_dim)
# set discriminator to non-trainanle 
discriminator.trainable = False
# compile the whole C-GAN architectu
cgan = cgan(generator, discriminator)
cgan.compile(loss='binary_crossentropy', optimizer=Adam())

ValueError: Exception encountered when calling layer 'concatenate_1' (type Concatenate).

Dimension 1 in both shapes must be equal, but are 28 and 700. Shapes are [?,28,28] and [?,700,460]. for '{{node model_2/concatenate_1/concat}} = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32](Placeholder, model_2/reshape_2/Reshape, model_2/concatenate_1/concat/axis)' with input shapes: [?,28,28,1], [?,700,460,1], [] and with computed input tensors: input[2] = <3>.

Call arguments received by layer 'concatenate_1' (type Concatenate):
  • inputs=['tf.Tensor(shape=(None, 28, 28, 1), dtype=float32)', 'tf.Tensor(shape=(None, 700, 460, 1), dtype=float32)']

In [ ]:
# import os
# import cv2

# def load_data(path1, path2):

#     files = []

#     paths = [path1, path2]

#     for path in paths:

#         for folder in os.listdir(path):

#             if "." not in folder: 

#                 for image in os.listdir(path + folder):

#                     img = cv2.imread(path + folder + "/" + image)

#                     files.append(np.array(img))

#     return files

import os
import cv2

def load_data():

    images = []

    path = 'D:/Medical_Imaging_Projekt/Dataset/benign/'

    for folder in os.listdir(path):

        if "." not in folder: 

            for image in os.listdir(path + folder):

                img = cv2.imread(path + folder + "/" + image)

                images.append(np.array(img))

    return images

In [ ]:
# label to category dictionary
dict_cancer = {0: "benign", 1: "malignant"}

# function to plot and save sample images
def plot_sample_images(epoch ,rows=5,columns=4):

    z = np.random.normal(0, 1, (rows * columns, z_dim))
    a =np.arange(0,10)
    b =np.arange(0,10)

    labels = np.append(a,b).reshape(-1,1)
    
    gen_imgs = generator.predict([z, labels])

    gen_imgs = 0.5 * gen_imgs + 0.5
    print("Epoch : %d "%(epoch+1))
    fig, axs = plt.subplots(rows,
                            columns,
                            figsize =(50, 20),
                            sharey=True,
                            sharex=True)

    cnt = 0
    for i in range(rows):
        for j in range(columns):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            axs[i, j].set_title("Type: %s" % dict_cancer.get(labels[cnt][0]))
            cnt += 1
    fig.savefig('image%d.jpg'%(epoch))

 
# define training step
def train(epochs, batch_size, sample_interval):

    #  import Fashion-MNIST dataset
    data = load_data()

    split = 0.8
    X_train = data[ : int(len(data) * split)]
    Y_train = data[int(len(data) * split) : ]

    X_train = X_train.astype("float32") / 255.0
    X_train = np.reshape(X_train, (-1, 28, 28, 1))


    real = np.ones((batch_size, 1))

    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs): 
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs, labels = X_train[idx], Y_train[idx]

        z = np.random.normal(0, 1, (batch_size, z_dim))
        # generate images from generator
        gen_imgs = generator.predict([z, labels])
        # pass real an generated images to the discriminator and ctrain on them
        d_loss_real = discriminator.train_on_batch([imgs, labels], real)
        d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        z = np.random.normal(0, 1, (batch_size, z_dim))

        labels = np.random.randint(0, n_class, batch_size).reshape(-1, 1)
   
        g_loss = cgan.train_on_batch([z, labels], real)

        if (epoch + 1) % sample_interval == 0:

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %(epoch + 1, d_loss[0], 100 * d_loss[1], g_loss))

            plot_sample_images(epoch+1)

In [ ]:
epochs = 10 #20000
batch_size = 128
sample_interval = 2 # 2000

train(epochs, batch_size, sample_interval)